In [1]:
%pylab inline
from ase.spacegroup import crystal
from ase.lattice.cubic import BodyCenteredCubic
from ase.visualize import view
from lammpsrun import LAMMPS, Prism
from ase.io import write
from ase.build import *

Populating the interactive namespace from numpy and matplotlib


## Creating the simulation particles with ASE

In [13]:
homepath = "D:\\Cloud Storage\\GitRepositories\\LAMMPS_Simulation\\"
homepath

'D:\\Cloud Storage\\GitRepositories\\LAMMPS_Simulation\\'

In [2]:
Fe_bulk = bulk('Fe', 'bcc', a=3.5, cubic = True) * 10
#BodyCenteredCubic(directions=[[1,0,0], [0,1,0], [0,0,1]], size=(2,2,3), 
#symbol='Fe', pbc=(1,1,1), latticeconstant=2.9)

In [4]:
Al_bulk = bulk('Al', 'fcc', a=34.0, cubic = True)

## Initial Simulation with Al

In [5]:
al_lammps_params = {
    # Initialization
    "units": "metal",
    "atom_style": "atomic",
    
    # Forcefield definition
    "pair_style": "eam/alloy",
    "pair_coeff": ['* * ' + 'Al99.eam.alloy' + ' Al'],
    "neighbor": "2.0 bin",
    "neighbor_modify": "delay 10 check yes",
    
    # Run and Minimization
    "minimize": "1e-25 1e-25 5000 10000",
    "min_style": "cg",
    "fix": ["1 all box/relax iso 0.0 vmax 0.001"]
}

In [16]:
al_calc = LAMMPS(tmp_dir = homepath + "temp", 
                 parameters = al_lammps_params,
                 keep_tmp_files=True, no_data_file=True)
Al_bulk.set_calculator(al_calc)
al_calc.atoms = Al_bulk.copy()
#calc.no_data_file = True
al_calc.prism = Prism(al_calc.atoms.get_cell())
al_calc.write_lammps_in(lammps_in = homepath + "input\\al_eam_minimize.in",
                        lammps_trj = homepath + "output\\al_eam_minimize.lammpstrj")

## Something else

In [3]:
lammps_params = {
    "run": "5000",
    "pair_style": "reax/c NULL",
    "pair_coeff": ['* * ' + '../reax_FeOH3/ffield.reax.Fe_O_C_H' + ' Fe'],
    "fix": ["1 all nve", "2 all qeq/reax 1 0.0 10.0 1e-6 reax/c"],
    "mass": ["1 55.845"],
    "timestep": "0.1"
}

In [4]:
calc = LAMMPS(tmp_dir="D:\\Yiming\\Desktop\\LAMMPS-TEST\\lammps_temp", 
              parameters = lammps_params,
              keep_tmp_files=True, no_data_file=True)
Fe_bulk.set_calculator(calc)
calc.atoms = Fe_bulk.copy()
calc.no_data_file = True
calc.prism = Prism(calc.atoms.get_cell())

In [5]:
calc.write_lammps_in(lammps_in = "D:\\Yiming\\Desktop\\LAMMPS-TEST\\lammps_temp\\test.in",
                     lammps_trj = "D:\\Yiming\\Desktop\\LAMMPS-TEST\\lammps_temp\\test.lammpstraj")